<a href="https://colab.research.google.com/github/Atulkhiyani0909/ByteVerse_NayaSetu/blob/main/Chatbot_RAG/NyayaSetu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [ ]:
%pip install --quiet --upgrade \
    langchain langchain-google-genai pymongo langchain_community \
    langchain_pinecone pinecone-client langchain_huggingface \
    numpy transformers --force-reinstall

## Environment

In [7]:
from google.colab import userdata
import os

In [8]:
os.environ['LANGCHAIN_TRACING_V2'] = userdata.get('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = userdata.get('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
os.environ['HUGGING_FACE_API_KEY']=userdata.get('HUGGING_FACE_API_KEY')

os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')
os.environ['MONGO_URL']=userdata.get('MONGO_URL')

## Importing Libraries

In [9]:
import os
from typing import List, Dict, Any, Optional,Literal,Union
from bson import ObjectId

# Core ML and RAG Libraries
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
from pymongo import MongoClient
from pydantic import BaseModel,Field
from functools import lru_cache
from langchain.load import dumps,loads

In [10]:
@lru_cache(maxsize=1)
def get_llm():
    return ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0)

llm = get_llm()

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

class language_detector(BaseModel):
    language: str = Field(..., description="Detected Language")
    translated: str = Field(..., description="Translated to English")

def query_to_english(query: str,memory: List) -> dict:
    """Detects the language of the input query and translates it to English."""

    lan_example = '''{
        "language": "Hindi",
        "translated": "Hello, how are you?"
    }'''

    prompt = """Translate the following query to clear English while preserving its context and intent.
    If the query is ambiguous, you can rephrase it, but do not change its original meaning utilize this Chat history to rewrite
    this ambigous query : {memory}

    Query: {query}

    Also, detect the language of the query and store it in "language".

    Output should strictly follow this format:
    {example}
    """

    llm3 = get_llm().with_structured_output(language_detector)

    trans_template = ChatPromptTemplate.from_template(
        template=prompt,
        partial_variables={
            'example': lan_example,
            'memory': memory
            }
    )

    trans_chain = trans_template | llm3  # No need for StrOutputParser since output is structured
    return trans_chain.invoke({'query': query}).model_dump()  # Ensure structured dict output

In [11]:
memory = [()]

In [12]:
query_to_english("Meri police ne complain nahi darj karwai",memory)

{'language': 'Hindi', 'translated': 'My police complaint was not registered'}